In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
from copy import deepcopy
import os

file_to_load = "Salary_by_region2.csv"

In [2]:
#read region data
region_data = pd.read_csv(file_to_load)

#Extract data by Region from the Master DF
Southern_Region = region_data.loc[region_data["Region"] == "Southern", :]

#List salary change over the lenght of mid-career by region; from school with hightest Starting_Mid to lowest Mid_90th
Southern_Sort = Southern_Region.sort_values(['Starting Median Salary', 'School Name'], ascending=[False, True])

#Clean DF to the areas of focus
South_Clean = Southern_Sort[["School Name", "Region", "Starting Median Salary", "Mid-Career 75th Percentile Salary", "Mid-Career 90th Percentile Salary"]]
South_Clean = South_Clean.dropna()

#Index data for Region Column
South_Clean.set_index('Region', inplace=True)

#Add unique index value row
df = South_Clean.reset_index()
df.head()

,Region,School Name,Starting Median Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Southern,Rice University,"$64,000.00","$141,000.00","$216,000.00"
1,Southern,Duke University,"$58,900.00","$149,000.00","$218,000.00"
2,Southern,Georgia Institute of Technology,"$58,300.00","$137,000.00","$183,000.00"
3,Southern,Georgetown University,"$55,000.00","$174,000.00","$248,000.00"
4,Southern,Virginia Polytechnic Institute and State Unive...,"$53,500.00","$124,000.00","$163,000.00"


In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
base_url = 'https://www.payscale.com/cost-of-living-calculator/'

In [6]:
#Create list of links to scrape 
city_list= ['Georgia-Atlanta', 'Texas-Austin','North-Carolina-Charlotte', 'Texas-Dallas',\
            'Texas-Houston', 'Florida-Miami', 'Tennessee-Nashville', 'Florida-Orlando', 'North-Carolina-Raleigh', 'Florida-Tampa', 'District-of-Columbia-Washington']
count = 0
pages = []
for page in city_list:
    page_url = f"{base_url}{city_list[count]}"
    count = count + 1
    pages.append(page_url)
    print(page_url)
    
    

https://www.payscale.com/cost-of-living-calculator/Georgia-Atlanta
https://www.payscale.com/cost-of-living-calculator/Texas-Austin
https://www.payscale.com/cost-of-living-calculator/North-Carolina-Charlotte
https://www.payscale.com/cost-of-living-calculator/Texas-Dallas
https://www.payscale.com/cost-of-living-calculator/Texas-Houston
https://www.payscale.com/cost-of-living-calculator/Florida-Miami
https://www.payscale.com/cost-of-living-calculator/Tennessee-Nashville
https://www.payscale.com/cost-of-living-calculator/Florida-Orlando
https://www.payscale.com/cost-of-living-calculator/North-Carolina-Raleigh
https://www.payscale.com/cost-of-living-calculator/Florida-Tampa
https://www.payscale.com/cost-of-living-calculator/District-of-Columbia-Washington


In [7]:
headers = ['Median Home Price','Median Rent','Energy Bill','Phone Bill','Gas','Loaf of Bread', \
            'Gallon of Milk','Carton of Eggs','Bunch of Bananas','Hamburger',"Doctor's Visit",\
            'Dentist Visit','Optometrist Visit','Rx Drug','Veterinary Visit']

In [8]:
count = 0
lifestyle_df = pd.DataFrame()
for page in pages:
    browser.visit(page)
    south_region = browser.html
    southern_soup = bs(south_region, 'html.parser')
    values = []
    for info in southern_soup.find_all('div', class_='itemized__item-container'):   
        for v in info.find_all('div', class_='itemized__item-value'):
            value = v.text
            values.append(deepcopy(value))
        
    style_df = pd.DataFrame([{headers[x]:values[x] for x in range(len(headers))}])
    lifestyle_df = lifestyle_df.append(deepcopy(style_df))  
    

In [9]:
df = pd.DataFrame(lifestyle_df)
df['States']= city_list


In [10]:
Southern_data = df.set_index('States')
Southern_data.head()

,Median Home Price,Median Rent,Energy Bill,Phone Bill,Gas,Loaf of Bread,Gallon of Milk,Carton of Eggs,Bunch of Bananas,Hamburger,Doctor's Visit,Dentist Visit,Optometrist Visit,Rx Drug,Veterinary Visit
States,,,,,,,,,,,,,,,
Georgia-Atlanta,"$327,568","$1,026 / month",$143.01 / month,$152.45 / month,$2.64 / gallon,$3.50,$2.04,$1.90,$3.53,$4.46,$119.01,$102.24,$111.52,$474.08,$52.50
Texas-Austin,"$353,246","$1,107 / month",$147.04 / month,$156.74 / month,$2.44 / gallon,$2.92,$1.70,$1.59,$2.95,$3.72,$110.05,$94.54,$103.12,$438.35,$51.88
North-Carolina-Charlotte,"$296,685",$929.39 / month,$153.91 / month,$164.06 / month,$2.53 / gallon,$3.12,$1.82,$1.70,$3.15,$3.98,$114.14,$98.06,$106.96,$454.67,$52.34
Texas-Dallas,"$364,003","$1,140 / month",$174.87 / month,$186.40 / month,$2.59 / gallon,$3.16,$1.84,$1.72,$3.19,$4.02,$112.04,$96.25,$104.98,$446.29,$52.65
Texas-Houston,"$363,656","$1,139 / month",$185.43 / month,$197.66 / month,$2.54 / gallon,$2.89,$1.68,$1.57,$2.92,$3.68,$100.97,$86.74,$94.61,$402.19,$48.64


In [11]:
for x in Southern_data:
    Southern_data[x] = Southern_data[x].str.replace("$","")
    Southern_data[x] = Southern_data[x].str.replace("month","")
    Southern_data[x] = Southern_data[x].str.replace("/","")
    Southern_data[x] = Southern_data[x].str.replace("gallon","")
    Southern_data[x] = Southern_data[x].str.replace(",","")
    Southern_data[x] = pd.to_numeric(Southern_data[x])

Southern_data.head()

,Median Home Price,Median Rent,Energy Bill,Phone Bill,Gas,Loaf of Bread,Gallon of Milk,Carton of Eggs,Bunch of Bananas,Hamburger,Doctor's Visit,Dentist Visit,Optometrist Visit,Rx Drug,Veterinary Visit
States,,,,,,,,,,,,,,,
Georgia-Atlanta,327568,1026.00,143.01,152.45,2.64,3.50,2.04,1.90,3.53,4.46,119.01,102.24,111.52,474.08,52.50
Texas-Austin,353246,1107.00,147.04,156.74,2.44,2.92,1.70,1.59,2.95,3.72,110.05,94.54,103.12,438.35,51.88
North-Carolina-Charlotte,296685,929.39,153.91,164.06,2.53,3.12,1.82,1.70,3.15,3.98,114.14,98.06,106.96,454.67,52.34
Texas-Dallas,364003,1140.00,174.87,186.40,2.59,3.16,1.84,1.72,3.19,4.02,112.04,96.25,104.98,446.29,52.65
Texas-Houston,363656,1139.00,185.43,197.66,2.54,2.89,1.68,1.57,2.92,3.68,100.97,86.74,94.61,402.19,48.64


In [12]:
Southern_data.columns = ['STATE','MORTGAGE','ELECTRICITY','PHONE','GAS','BREAD','MILK','EGGS','BANANAS','POLTRY','PHYSICIAN', 'DENTIST','OPTOMISTRIST', 'RX','VET']
Southern_data['HLTH & WELL']=Southern_data['PHYSICIAN'] + Southern_data['DENTIST'] + Southern_data['OPTOMISTRIST'] + Southern_data['RX']
Southern_data['EAT-IN']=Southern_data['BREAD'] + Southern_data['MILK'] + Southern_data['BANANAS'] + Southern_data['POLTRY']
Southern_data = Southern_data.drop(['BREAD','MILK','EGGS','BANANAS','POLTRY','PHYSICIAN', 'DENTIST', 'OPTOMISTRIST','RX'], axis=1)
Southern_data


,STATE,MORTGAGE,ELECTRICITY,PHONE,GAS,VET,HLTH & WELL,EAT-IN
States,,,,,,,,
Georgia-Atlanta,327568,1026.00,143.01,152.45,2.64,52.50,806.85,13.53
Texas-Austin,353246,1107.00,147.04,156.74,2.44,51.88,746.06,11.29
North-Carolina-Charlotte,296685,929.39,153.91,164.06,2.53,52.34,773.83,12.07
Texas-Dallas,364003,1140.00,174.87,186.40,2.59,52.65,759.56,12.21
Texas-Houston,363656,1139.00,185.43,197.66,2.54,48.64,684.51,11.17
Florida-Miami,498986,1563.00,165.31,176.22,2.93,52.19,728.05,13.56
Tennessee-Nashville,316117,990.26,150.73,160.67,2.62,53.37,613.21,12.48
Florida-Orlando,308483,966.34,186.44,198.74,2.45,48.69,662.00,13.14
North-Carolina-Raleigh,241512,756.55,161.62,172.29,2.39,54.82,772.33,14.57


In [13]:
Southern_data.mean(axis = 0)

STATE          381542.272727
MORTGAGE         1195.221818
ELECTRICITY       165.952727
PHONE             176.900000
GAS                 2.602727
VET                51.586364
HLTH & WELL       728.045455
EAT-IN             12.821818
dtype: float64

In [14]:
#Transpose the DataFrame
southern_avg = pd.DataFrame(Southern_data.mean())
southern_avg_t = southern_avg.T
southern_avg_t 

,STATE,MORTGAGE,ELECTRICITY,PHONE,GAS,VET,HLTH & WELL,EAT-IN
0,381542.272727,1195.221818,165.952727,176.9,2.602727,51.586364,728.045455,12.821818
